In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

In [2]:
# Replace 'products.csv' with your actual file name
df = pd.read_csv("SampleData.csv")

# Check first few rows
df.head()

,MappingModeName,ServiceAndProductMappingId,SourceMasterBrand,SourceBrand,SourceSubBrand,SourceCategory,SourceSubcategory,SourceDescription,SourcePackagingTypeName,SourceSize,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,Map,6242009,NaN,L'OREAL,Styling Products,Unspecified,Unspecified,Curl Expression Mousse,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Map,6324365,NaN,NaN,NaN,Other,NaN,Loreal Curl Expression Mousse,Not Specified,8.2 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Map,6326141,NaN,NaN,NaN,Styling,NaN,Curls 10 In 1 Mousse,Not Specified,8.5 Oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Map,6182020,NaN,L'OREAL PROFESSIONNEL,CURL EXPRESSION,Unspecified,Unspecified,SOIN MULTI BENEFICES CREME-MOUSSE,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Map,6493784,NaN,NaN,NaN,Styling,NaN,L'Oreal Curl Expression Mousse,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Select the columns to compare
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']

# Fill missing values with empty strings to avoid errors
df_selected = df[cols].fillna("")

In [4]:
# Combine all columns into one string per row
combined_text = df_selected.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [5]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(combined_text)

In [6]:
# eps and min_samples can be tuned
model = DBSCAN(eps=0.5, min_samples=3, metric='cosine')
labels = model.fit_predict(X)

# Add cluster labels to the original dataframe
df['Cluster'] = labels

In [7]:
# View how many clusters were formed (excluding noise)
import numpy as np
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Number of clusters found: {n_clusters}")

# Show some examples from each cluster
df.groupby('Cluster').head(3)

Number of clusters found: 6


,MappingModeName,ServiceAndProductMappingId,SourceMasterBrand,SourceBrand,SourceSubBrand,SourceCategory,SourceSubcategory,SourceDescription,SourcePackagingTypeName,SourceSize,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Cluster
0,Map,6242009,NaN,L'OREAL,Styling Products,Unspecified,Unspecified,Curl Expression Mousse,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Map,6324365,NaN,NaN,NaN,Other,NaN,Loreal Curl Expression Mousse,Not Specified,8.2 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,Map,6326141,NaN,NaN,NaN,Styling,NaN,Curls 10 In 1 Mousse,Not Specified,8.5 Oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,Map,6182020,NaN,L'OREAL PROFESSIONNEL,CURL EXPRESSION,Unspecified,Unspecified,SOIN MULTI BENEFICES CREME-MOUSSE,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
26,Map,7581323,NaN,L'Oréal Professionnel revente,Curl expression,Unspecified,Unspecified,Curl Expression Crème-en-Mousse 10-en-1 250 ml,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
28,Map,6182015,NaN,L'Oreal,CURL EXPRESSION,Unspecified,Unspecified,CREME EN MOUSSE,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
41,Map,1431510,NaN,NaN,NaN,Conditioner,NaN,Stimulate me rinse,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
42,Map,9258707,NaN,NaN,NaN,a7.Products Hair,NaN,KM 44 stimulate me wash 8.4 oz,Not Specified,8.4 oz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
43,Map,1700616,NaN,NaN,NaN,Wash,NaN,Kevin Murphy Stimulate Me Wash,Not Specified,8.4oz.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
130,Map,5064219,NaN,NaN,NaN,Shampoo,NaN,km stimulte wash,Not Specified,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [8]:
df.to_csv("clustered_productsnew.csv", index=False)
print("Clustering complete. Output saved to 'clustered_products.csv'")

Clustering complete. Output saved to 'clustered_products.csv'


In [1]:
# 📦 STEP 1: Install Required Libraries (Run this in a separate cell if needed)
# !pip install pandas sentence-transformers hdbscan scikit-learn

# 📥 STEP 2: Load and Prepare Data
import pandas as pd

df = pd.read_csv("products.csv")  # Replace with your actual filename
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']  # excluding barcode for better clustering
df = df[cols].fillna("")

# Combine selected columns into one text column
df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# 🧼 STEP 3: Clean the Text
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

df['text'] = df['text'].apply(clean_text)

# 🤖 STEP 4: Generate Sentence Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast, 384-dim
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 🧠 STEP 5: Run HDBSCAN Clustering
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['Cluster'] = labels

# 🧪 STEP 6: Evaluate Clustering Quality
from sklearn.metrics import silhouette_score

# Filter out noise (-1 labels) for scoring
if len(set(labels)) > 1 and (labels != -1).sum() > 1:
    score = silhouette_score(embeddings[labels != -1], labels[labels != -1])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Not enough clusters to compute silhouette score.")

# 📤 STEP 7: Export Clustered Data
df.to_csv("clustered_products_new.csv", index=False)
print("✅ Clustering complete. Results saved to 'clustered_products.csv'.")

# 👀 STEP 8: Preview a Few Rows from Each Cluster
df.groupby('Cluster').head(3)


FileNotFoundError: [Errno 2] No such file or directory: 'products.csv'

In [2]:
# 📦 STEP 1: Install Required Libraries (Run this in a separate cell if needed)
# !pip install pandas sentence-transformers hdbscan scikit-learn

# 📥 STEP 2: Load and Prepare Data
import pandas as pd

df = pd.read_csv("SampleData.csv")  # Replace with your actual filename
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']  # excluding barcode for better clustering
df = df[cols].fillna("")

# Combine selected columns into one text column
df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# 🧼 STEP 3: Clean the Text
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

df['text'] = df['text'].apply(clean_text)

# 🤖 STEP 4: Generate Sentence Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast, 384-dim
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 🧠 STEP 5: Run HDBSCAN Clustering
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['Cluster'] = labels

# 🧪 STEP 6: Evaluate Clustering Quality
from sklearn.metrics import silhouette_score

# Filter out noise (-1 labels) for scoring
if len(set(labels)) > 1 and (labels != -1).sum() > 1:
    score = silhouette_score(embeddings[labels != -1], labels[labels != -1])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Not enough clusters to compute silhouette score.")

# 📤 STEP 7: Export Clustered Data
df.to_csv("clustered_products.csv", index=False)
print("✅ Clustering complete. Results saved to 'clustered_products.csv'.")

# 👀 STEP 8: Preview a Few Rows from Each Cluster
df.groupby('Cluster').head(3)


ModuleNotFoundError: No module named 'sentence_transformers'

In [3]:
# 📦 STEP 1: Install Required Libraries (Run this in a separate cell if needed)
pip install pandas sentence-transformers hdbscan scikit-learn

# 📥 STEP 2: Load and Prepare Data
import pandas as pd

df = pd.read_csv("products.csv")  # Replace with your actual filename
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']  # excluding barcode for better clustering
df = df[cols].fillna("")

# Combine selected columns into one text column
df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# 🧼 STEP 3: Clean the Text
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

df['text'] = df['text'].apply(clean_text)

# 🤖 STEP 4: Generate Sentence Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast, 384-dim
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 🧠 STEP 5: Run HDBSCAN Clustering
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['Cluster'] = labels

# 🧪 STEP 6: Evaluate Clustering Quality
from sklearn.metrics import silhouette_score

# Filter out noise (-1 labels) for scoring
if len(set(labels)) > 1 and (labels != -1).sum() > 1:
    score = silhouette_score(embeddings[labels != -1], labels[labels != -1])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Not enough clusters to compute silhouette score.")

# 📤 STEP 7: Export Clustered Data
df.to_csv("clustered_products_new.csv", index=False)
print("✅ Clustering complete. Results saved to 'clustered_products.csv'.")

# 👀 STEP 8: Preview a Few Rows from Each Cluster
df.groupby('Cluster').head(3)


SyntaxError: invalid syntax (1250447445.py, line 2)

In [ ]:
!pip install sentence-transformers

In [1]:

# 📥 STEP 2: Load and Prepare Data
import pandas as pd

df = pd.read_csv("products.csv")  # Replace with your actual filename
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']  # excluding barcode for better clustering
df = df[cols].fillna("")

# Combine selected columns into one text column
df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# 🧼 STEP 3: Clean the Text
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

df['text'] = df['text'].apply(clean_text)

# 🤖 STEP 4: Generate Sentence Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast, 384-dim
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 🧠 STEP 5: Run HDBSCAN Clustering
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['Cluster'] = labels

# 🧪 STEP 6: Evaluate Clustering Quality
from sklearn.metrics import silhouette_score

# Filter out noise (-1 labels) for scoring
if len(set(labels)) > 1 and (labels != -1).sum() > 1:
    score = silhouette_score(embeddings[labels != -1], labels[labels != -1])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Not enough clusters to compute silhouette score.")

# 📤 STEP 7: Export Clustered Data
df.to_csv("clustered_products_new.csv", index=False)
print("✅ Clustering complete. Results saved to 'clustered_products.csv'.")

# 👀 STEP 8: Preview a Few Rows from Each Cluster
df.groupby('Cluster').head(3)


FileNotFoundError: [Errno 2] No such file or directory: 'products.csv'

In [3]:
# 📦 STEP 1: Install Required Libraries (Run this in a separate cell if needed)
!pip install pandas sentence-transformers hdbscan scikit-learn



# 📥 STEP 2: Load and Prepare Data
import pandas as pd

df = pd.read_csv("SampleData.csv")  # Replace with your actual filename
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']  # excluding barcode for better clustering
df = df[cols].fillna("")

# Combine selected columns into one text column
df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# 🧼 STEP 3: Clean the Text
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

df['text'] = df['text'].apply(clean_text)

# 🤖 STEP 4: Generate Sentence Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast, 384-dim
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 🧠 STEP 5: Run HDBSCAN Clustering
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['Cluster'] = labels

# 🧪 STEP 6: Evaluate Clustering Quality
from sklearn.metrics import silhouette_score

# Filter out noise (-1 labels) for scoring
if len(set(labels)) > 1 and (labels != -1).sum() > 1:
    score = silhouette_score(embeddings[labels != -1], labels[labels != -1])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Not enough clusters to compute silhouette score.")

# 📤 STEP 7: Export Clustered Data
df.to_csv("clustered_products_new.csv", index=False)
print("✅ Clustering complete. Results saved to 'clustered_products.csv'.")

# 👀 STEP 8: Preview a Few Rows from Each Cluster
df.groupby('Cluster').head(3)


  Using cached sentence_transformers-5.0.0-py3-none-any.whl.metadata (16 kB)
     ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
     ---------- ----------------------------- 1.8/6.9 MB 9.9 MB/s eta 0:00:01
     ---------------------- ----------------- 3.9/6.9 MB 9.7 MB/s eta 0:00:01
     ---------------------------------- ----- 6.0/6.9 MB 9.8 MB/s eta 0:00:01
     ---------------------------------------- 6.9/6.9 MB 8.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached transformers-4.54.1-py3-none-any.whl.metadata (41 kB)
  Using cached huggingface_hub-0.34.3-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [39 lines of output]
  C:\Users\Sarthak.Prakash\AppData\Local\Temp\pip-build-env-3um73yq1\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
    warnings.warn(msg)
  C:\Users\Sarthak.Prakash\AppData\Local\Temp\pip-build-env-3um73yq1\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
    warnings.warn(msg)
  C:\Users\Sarthak.Prakash\AppData\Local\Temp\pip-build-env-3um73yq1\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
  !!
  
          ********************************************************************************
          Please consider removing the following classifiers in favor of a SPDX license expression:
  
          License :: OSI Approved
  
          See https://packaging.python.org/en/latest/guides/writing-pyproje

ModuleNotFoundError: No module named 'sentence_transformers'

In [1]:
# 📥 STEP 2: Load and Prepare Data
import pandas as pd

df = pd.read_csv("SampleData.csv")  # Replace with your actual filename
cols = ['SourceDescription', 'SourceMasterBrand', 'SourceBrand']  # excluding barcode for better clustering
df = df[cols].fillna("")

# Combine selected columns into one text column
df['text'] = df.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

# 🧼 STEP 3: Clean the Text
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # normalize spaces
    return text

df['text'] = df['text'].apply(clean_text)

# 🤖 STEP 4: Generate Sentence Embeddings
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Small & fast, 384-dim
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True)

# 🧠 STEP 5: Run HDBSCAN Clustering
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

df['Cluster'] = labels

# 🧪 STEP 6: Evaluate Clustering Quality
from sklearn.metrics import silhouette_score

# Filter out noise (-1 labels) for scoring
if len(set(labels)) > 1 and (labels != -1).sum() > 1:
    score = silhouette_score(embeddings[labels != -1], labels[labels != -1])
    print(f"Silhouette Score (excluding noise): {score:.4f}")
else:
    print("Not enough clusters to compute silhouette score.")

# 📤 STEP 7: Export Clustered Data
df.to_csv("clustered_products_new.csv", index=False)
print("✅ Clustering complete. Results saved to 'clustered_products.csv'.")

# 👀 STEP 8: Preview a Few Rows from Each Cluster
df.groupby('Cluster').head(3)


ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your data
data = [
    "Curl Expression Mousse",
    "Loreal Curl Expression Mousse",
    "Curls 10 In 1 Mousse",
    "SOIN MULTI BENEFICES CREME-MOUSSE",
    "L'Oreal Curl Expression Mousse",
    "Curl Expression 10-In-1 Cream-In-Mousse",
    "Curl Expression - Mousse",
    "Curl epxpression 10 in 1 cream in mousse",
    "Curl Expression 10-IN-1",
    "Curl Expression 10 in 1 Mousse",
    "10 -in- 1 Professional Cream in Mouse",
    "Curl Expression Mousse",
    "oreal curl expression",
    "Curl Express Mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-in-Mousse",
    "Curl Expression Cream Mousse",
    "L'Oreal Curl Expression Mousse",
    "Curl Exp 10-1 mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Express Mousse",
    "Curl Expression Cream In Mouse",
    "L'Oreal Curl Expression - Curl Mousse",
    "Curl Expression Crème-en-Mousse 10-en-1 250 ml",
    "SERIE - Curl Expression - Mousse",
    "CREME EN MOUSSE",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-Mousse",
    "CURL EXPRESS MOUSSE",
    "Curl Expression Mousse 10-in-1",
    "Curl Expression 10 - 1 Mousse 8.2oz",
    "LP Curls 10 in 1 Mousse",
    "curl expression mousse",
    "loreal curl expressions mousse",
    "Curl Expressions 10-in-1 Cream Mousse",
    "Curl Expressions 10-IN-1 Cream-In-Mousse",
    "Loreal Creme Mouse",
    "Mousse 10 en 1 soin multi-bénéfiste"
]

df = pd.DataFrame(data, columns=['SourceDescription'])

# Canonical master product
master_product = "10 IN 1 CREAM IN MOUSSE"

# Load SBERT model (small and fast variant)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all descriptions and the master product
embeddings = model.encode(df['SourceDescription'].tolist(), convert_to_tensor=True)
master_embedding = model.encode(master_product, convert_to_tensor=True)

# Compute cosine similarity
similarities = util.cos_sim(master_embedding, embeddings)[0]

# Add similarity scores to DataFrame
df['SimilarityScore'] = similarities.cpu().numpy()

# Set a threshold to confirm match (optional)
threshold = 0.6
df['MatchedToMaster'] = df['SimilarityScore'] >= threshold
df['MappedProduct'] = df['MatchedToMaster'].apply(lambda x: master_product if x else "Check manually")

# View result
import caas_jupyter_tools as cjtools
cjtools.display_dataframe_to_user(name="Matched Products", dataframe=df)


ModuleNotFoundError: No module named 'sentence_transformers'

In [3]:
!pip install -q sentence-transformers

In [4]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your data
data = [
    "Curl Expression Mousse",
    "Loreal Curl Expression Mousse",
    "Curls 10 In 1 Mousse",
    "SOIN MULTI BENEFICES CREME-MOUSSE",
    "L'Oreal Curl Expression Mousse",
    "Curl Expression 10-In-1 Cream-In-Mousse",
    "Curl Expression - Mousse",
    "Curl epxpression 10 in 1 cream in mousse",
    "Curl Expression 10-IN-1",
    "Curl Expression 10 in 1 Mousse",
    "10 -in- 1 Professional Cream in Mouse",
    "Curl Expression Mousse",
    "oreal curl expression",
    "Curl Express Mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-in-Mousse",
    "Curl Expression Cream Mousse",
    "L'Oreal Curl Expression Mousse",
    "Curl Exp 10-1 mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Express Mousse",
    "Curl Expression Cream In Mouse",
    "L'Oreal Curl Expression - Curl Mousse",
    "Curl Expression Crème-en-Mousse 10-en-1 250 ml",
    "SERIE - Curl Expression - Mousse",
    "CREME EN MOUSSE",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-Mousse",
    "CURL EXPRESS MOUSSE",
    "Curl Expression Mousse 10-in-1",
    "Curl Expression 10 - 1 Mousse 8.2oz",
    "LP Curls 10 in 1 Mousse",
    "curl expression mousse",
    "loreal curl expressions mousse",
    "Curl Expressions 10-in-1 Cream Mousse",
    "Curl Expressions 10-IN-1 Cream-In-Mousse",
    "Loreal Creme Mouse",
    "Mousse 10 en 1 soin multi-bénéfiste"
]

df = pd.DataFrame(data, columns=['SourceDescription'])

# Canonical master product
master_product = "10 IN 1 CREAM IN MOUSSE"

# Load SBERT model (small and fast variant)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all descriptions and the master product
embeddings = model.encode(df['SourceDescription'].tolist(), convert_to_tensor=True)
master_embedding = model.encode(master_product, convert_to_tensor=True)

# Compute cosine similarity
similarities = util.cos_sim(master_embedding, embeddings)[0]

# Add similarity scores to DataFrame
df['SimilarityScore'] = similarities.cpu().numpy()

# Set a threshold to confirm match (optional)
threshold = 0.6
df['MatchedToMaster'] = df['SimilarityScore'] >= threshold
df['MappedProduct'] = df['MatchedToMaster'].apply(lambda x: master_product if x else "Check manually")

# View result
import caas_jupyter_tools as cjtools
cjtools.display_dataframe_to_user(name="Matched Products", dataframe=df)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sarthak.Prakash\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


ModuleNotFoundError: No module named 'caas_jupyter_tools'

In [5]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your data
data = [
    "Curl Expression Mousse",
    "Loreal Curl Expression Mousse",
    "Curls 10 In 1 Mousse",
    "SOIN MULTI BENEFICES CREME-MOUSSE",
    "L'Oreal Curl Expression Mousse",
    "Curl Expression 10-In-1 Cream-In-Mousse",
    "Curl Expression - Mousse",
    "Curl epxpression 10 in 1 cream in mousse",
    "Curl Expression 10-IN-1",
    "Curl Expression 10 in 1 Mousse",
    "10 -in- 1 Professional Cream in Mouse",
    "Curl Expression Mousse",
    "oreal curl expression",
    "Curl Express Mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-in-Mousse",
    "Curl Expression Cream Mousse",
    "L'Oreal Curl Expression Mousse",
    "Curl Exp 10-1 mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Express Mousse",
    "Curl Expression Cream In Mouse",
    "L'Oreal Curl Expression - Curl Mousse",
    "Curl Expression Crème-en-Mousse 10-en-1 250 ml",
    "SERIE - Curl Expression - Mousse",
    "CREME EN MOUSSE",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-Mousse",
    "CURL EXPRESS MOUSSE",
    "Curl Expression Mousse 10-in-1",
    "Curl Expression 10 - 1 Mousse 8.2oz",
    "LP Curls 10 in 1 Mousse",
    "curl expression mousse",
    "loreal curl expressions mousse",
    "Curl Expressions 10-in-1 Cream Mousse",
    "Curl Expressions 10-IN-1 Cream-In-Mousse",
    "Loreal Creme Mouse",
    "Mousse 10 en 1 soin multi-bénéfiste"
]

df = pd.DataFrame(data, columns=['SourceDescription'])

# Canonical master product
master_product = "Curl Expression 10-in-1 Cream-in-Mousse"

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode
embeddings = model.encode(df['SourceDescription'].tolist(), convert_to_tensor=True)
master_embedding = model.encode(master_product, convert_to_tensor=True)

# Similarity
similarities = util.cos_sim(master_embedding, embeddings)[0]
df['SimilarityScore'] = similarities.cpu().numpy()

# Mapping logic
threshold = 0.6
df['MatchedToMaster'] = df['SimilarityScore'] >= threshold
df['MappedProduct'] = df['MatchedToMaster'].apply(lambda x: master_product if x else "Check manually")

# Show result
print(df[['SourceDescription', 'SimilarityScore', 'MappedProduct']])


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


                                 SourceDescription  SimilarityScore  \
0                           Curl Expression Mousse         0.819024   
1                    Loreal Curl Expression Mousse         0.792863   
2                             Curls 10 In 1 Mousse         0.722310   
3                SOIN MULTI BENEFICES CREME-MOUSSE         0.457562   
4                   L'Oreal Curl Expression Mousse         0.769618   
5          Curl Expression 10-In-1 Cream-In-Mousse         1.000000   
6                         Curl Expression - Mousse         0.810031   
7         Curl epxpression 10 in 1 cream in mousse         0.844786   
8                          Curl Expression 10-IN-1         0.718177   
9                   Curl Expression 10 in 1 Mousse         0.850729   
10           10 -in- 1 Professional Cream in Mouse         0.345365   
11                          Curl Expression Mousse         0.819024   
12                           oreal curl expression         0.635643   
13    

In [6]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load your data
data = [
    "Curl Expression Mousse",
    "Loreal Curl Expression Mousse",
    "Curls 10 In 1 Mousse",
    "SOIN MULTI BENEFICES CREME-MOUSSE",
    "L'Oreal Curl Expression Mousse",
    "Curl Expression 10-In-1 Cream-In-Mousse",
    "Curl Expression - Mousse",
    "Curl epxpression 10 in 1 cream in mousse",
    "Curl Expression 10-IN-1",
    "Curl Expression 10 in 1 Mousse",
    "10 -in- 1 Professional Cream in Mouse",
    "Curl Expression Mousse",
    "oreal curl expression",
    "Curl Express Mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-in-Mousse",
    "Curl Expression Cream Mousse",
    "L'Oreal Curl Expression Mousse",
    "Curl Exp 10-1 mousse",
    "Curl Expression Mousse",
    "Curl Expression Mousse",
    "Curl Express Mousse",
    "Curl Expression Cream In Mouse",
    "L'Oreal Curl Expression - Curl Mousse",
    "Curl Expression Crème-en-Mousse 10-en-1 250 ml",
    "SERIE - Curl Expression - Mousse",
    "CREME EN MOUSSE",
    "Curl Expression Mousse",
    "Curl Expression 10-in-1 Cream-Mousse",
    "CURL EXPRESS MOUSSE",
    "Curl Expression Mousse 10-in-1",
    "Curl Expression 10 - 1 Mousse 8.2oz",
    "LP Curls 10 in 1 Mousse",
    "curl expression mousse",
    "loreal curl expressions mousse",
    "Curl Expressions 10-in-1 Cream Mousse",
    "Curl Expressions 10-IN-1 Cream-In-Mousse",
    "Loreal Creme Mouse",
    "Mousse 10 en 1 soin multi-bénéfiste"
]

df = pd.DataFrame(data, columns=['SourceDescription'])

# Canonical master product
master_product = "10 IN 1 CREAM IN MOUSSE"

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode
embeddings = model.encode(df['SourceDescription'].tolist(), convert_to_tensor=True)
master_embedding = model.encode(master_product, convert_to_tensor=True)

# Similarity
similarities = util.cos_sim(master_embedding, embeddings)[0]
df['SimilarityScore'] = similarities.cpu().numpy()

# Mapping logic
threshold = 0.6
df['MatchedToMaster'] = df['SimilarityScore'] >= threshold
df['MappedProduct'] = df['MatchedToMaster'].apply(lambda x: master_product if x else "Check manually")

# Show result
print(df[['SourceDescription', 'SimilarityScore', 'MappedProduct']])


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


                                 SourceDescription  SimilarityScore  \
0                           Curl Expression Mousse         0.292035   
1                    Loreal Curl Expression Mousse         0.310874   
2                             Curls 10 In 1 Mousse         0.625827   
3                SOIN MULTI BENEFICES CREME-MOUSSE         0.542094   
4                   L'Oreal Curl Expression Mousse         0.342276   
5          Curl Expression 10-In-1 Cream-In-Mousse         0.622831   
6                         Curl Expression - Mousse         0.280588   
7         Curl epxpression 10 in 1 cream in mousse         0.686028   
8                          Curl Expression 10-IN-1         0.217655   
9                   Curl Expression 10 in 1 Mousse         0.445685   
10           10 -in- 1 Professional Cream in Mouse         0.576613   
11                          Curl Expression Mousse         0.292035   
12                           oreal curl expression         0.127491   
13    

In [8]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV file (update filename as needed)
df = pd.read_csv('SampleFullData.csv')  # Make sure it has 'SourceDescription' and 'ProductName' columns

# Drop NA and get unique lists
source_descriptions = df['SourceDescription'].dropna().unique().tolist()
master_products = df['ProductName'].dropna().unique().tolist()

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode source descriptions and master product names
source_embeddings = model.encode(source_descriptions, convert_to_tensor=True)
master_embeddings = model.encode(master_products, convert_to_tensor=True)

# Compute cosine similarities between each source and each master
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# For each source description, find best matching master
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Build result dataframe
matched_df = pd.DataFrame({
    'SourceDescription': source_descriptions,
    'BestMatchProduct': [master_products[i] for i in best_match_idx],
    'SimilarityScore': best_scores
})

# Optional: Filter low-confidence matches
threshold = 0.6
matched_df['Confidence'] = matched_df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Show results
print(matched_df.sort_values('SimilarityScore', ascending=False))
matched_df.to_csv('outputofmodel.csv', index=False)


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


          SourceDescription     BestMatchProduct  SimilarityScore Confidence
554         Zero Yellow Kit      ZERO YELLOW KIT         1.000000       High
428     RICH REPAIR SHAMPOO  RICH REPAIR SHAMPOO         1.000000       High
81        STIMULATE ME WASH    STIMULATE ME WASH         1.000000       High
83        stimulate me wash    STIMULATE ME WASH         1.000000       High
468     Rich repair shampoo  RICH REPAIR SHAMPOO         1.000000       High
..                      ...                  ...              ...        ...
475              Final Sale    HAIR DRESSERS SET         0.214029        Low
440     Wishes Do Come True   STIMULATE ME RINSE         0.203026        Low
550  Bb Wishes do come true    REVOLVE WOMEN DUO         0.186687        Low
254  Stim rins9339341003991   STIMULATE ME RINSE         0.168589        Low
482                  Bumble    STIMULATE ME WASH         0.142000        Low

[568 rows x 4 columns]


In [11]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV file
df = pd.read_csv('SampleFullData.csv')  # Must contain 'SourceDescription' and 'ProductName'

# Drop rows with missing data
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Get master product list (unique ProductName values)
master_products = df['ProductName'].dropna().unique().tolist()

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode master products
master_embeddings = model.encode(master_products, convert_to_tensor=True)

# Encode source descriptions (row-wise)
source_embeddings = model.encode(df['SourceDescription'].tolist(), convert_to_tensor=True)

# Compute cosine similarity for each row with all master products
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Get best match index and score for each source description
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Assign results back to dataframe
df['BestMatchProduct'] = [master_products[i] for i in best_match_idx]
df['SimilarityScore'] = best_scores

# Confidence label
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save output
df.to_csv('outputofmodel.csv', index=False)

# Preview
print("✅ Matching complete. Output saved to 'outputofmodel.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence']].head())


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\2082870063.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')  # Must contain 'SourceDescription' and 'ProductName'
C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Matching complete. Output saved to 'outputofmodel.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

            BestMatchProduct  SimilarityScore Confidence  
0       CURL DEFINING MOUSSE         0.913944       High  
1       CURL DEFINING MOUSSE         0.809130       High  
2  FRESH CURLS SPRING MOUSSE         0.802155       High  
3      MULTICROISSANCE CREME         0.684465       High  
4                CURL MOUSSE         0.805791       High  


In [13]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in helper columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']:
    df[col] = df[col].fillna('')

# Create enriched text for raw data
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'].astype(str) + ' | ' +
    'Brand: ' + df['SourceBrand'] + ' | ' +
    'SubBrand: ' + df['SourceSubBrand'] + ' | ' +
    'MasterBrand: ' + df['SourceMasterBrand']
)

# Create master list with additional brand info
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')

# Create enriched master product text
master_df['CombinedMasterText'] = (
    'Description: ' + master_df['ProductName'].astype(str) + ' | ' +
    'Brand: ' + master_df['BrandName'] + ' | ' +
    'SubBrand: ' + master_df['SubBrandName'] + ' | ' +
    'MasterBrand: ' + master_df['MasterBrandName']
)

# Encode with SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True)
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Get best matches
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Add matched info to df
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save to CSV
df.to_csv('outputofmodel1.csv', index=False)

# Preview
print("✅ Matching complete. Output saved to 'outputofmodel1.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence']].head())


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\3031593436.py:5: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')
C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Matching complete. Output saved to 'outputofmodel1.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

                BestMatchProduct  SimilarityScore Confidence  
0                    CURL MOUSSE         0.857571       High  
1                    CURL MOUSSE         0.828772       High  
2      CURLS FIRM STYLING MOUSSE         0.821535       High  
3        10 IN 1 CREAM IN MOUSSE         0.766348       High  
4  NATURAL CURL ENHANCING MOUSSE         0.820982       High  


In [14]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop incomplete rows
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Fill missing helper columns
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']:
    df[col] = df[col].fillna('')
for col in ['BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Create combined source text
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'].astype(str) + ' | ' +
    'Brand: ' + df['SourceBrand'] + ' | ' +
    'SubBrand: ' + df['SourceSubBrand'] + ' | ' +
    'MasterBrand: ' + df['SourceMasterBrand']
)

# Get master product list with unique entries
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()

# Create combined master text
master_df['CombinedMasterText'] = (
    'Description: ' + master_df['ProductName'].astype(str) + ' | ' +
    'Brand: ' + master_df['BrandName'] + ' | ' +
    'SubBrand: ' + master_df['SubBrandName'] + ' | ' +
    'MasterBrand: ' + master_df['MasterBrandName']
)

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode master product texts
master_embeddings_all = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True)
master_texts_all = master_df['CombinedMasterText'].tolist()
master_products_all = master_df['ProductName'].tolist()

# Store final matched values
best_matches = []
match_types = []
similarity_scores = []
matched_brands = []
matched_subbrands = []
matched_masterbrands = []

# Main loop: match each row individually
for idx, row in df.iterrows():
    source_text = row['CombinedSourceText']
    source_brand = row['SourceBrand'].strip().lower()
    source_masterbrand = row['SourceMasterBrand'].strip().lower()

    # Try to filter master products by brand or masterbrand
    filtered_master_df = master_df[
        (master_df['BrandName'].str.strip().str.lower() == source_brand) |
        (master_df['MasterBrandName'].str.strip().str.lower() == source_masterbrand)
    ]

    if not filtered_master_df.empty:
        # Encode only filtered master entries
        filtered_embeddings = model.encode(filtered_master_df['CombinedMasterText'].tolist(), convert_to_tensor=True)
        source_embedding = model.encode(source_text, convert_to_tensor=True)
        similarities = util.cos_sim(source_embedding, filtered_embeddings)[0]
        best_idx = int(similarities.argmax())
        score = float(similarities[best_idx])

        best_matches.append(filtered_master_df.iloc[best_idx]['ProductName'])
        matched_brands.append(filtered_master_df.iloc[best_idx]['BrandName'])
        matched_subbrands.append(filtered_master_df.iloc[best_idx]['SubBrandName'])
        matched_masterbrands.append(filtered_master_df.iloc[best_idx]['MasterBrandName'])
        match_types.append("BrandFiltered")
        similarity_scores.append(score)
    else:
        # Fallback to full master list
        source_embedding = model.encode(source_text, convert_to_tensor=True)
        similarities = util.cos_sim(source_embedding, master_embeddings_all)[0]
        best_idx = int(similarities.argmax())
        score = float(similarities[best_idx])

        best_matches.append(master_products_all[best_idx])
        matched_brands.append(master_df.iloc[best_idx]['BrandName'])
        matched_subbrands.append(master_df.iloc[best_idx]['SubBrandName'])
        matched_masterbrands.append(master_df.iloc[best_idx]['MasterBrandName'])
        match_types.append("FallbackGlobal")
        similarity_scores.append(score)

# Add results to original DataFrame
df['BestMatchProduct'] = best_matches
df['MatchedBrand'] = matched_brands
df['MatchedSubBrand'] = matched_subbrands
df['MatchedMasterBrand'] = matched_masterbrands
df['MatchType'] = match_types
df['SimilarityScore'] = similarity_scores
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= 0.6 else 'Low')
df['IsMismatch'] = df['ProductName'] != df['BestMatchProduct']

# Save to CSV
df.to_csv('outputofmodel3.csv', index=False)

# Summary
print("✅ Smart matching complete. Output saved to 'outputofmodel3.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'MatchType', 'SimilarityScore', 'Confidence', 'IsMismatch']].head())


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\1683330020.py:6: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')
C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


KeyboardInterrupt: 

In [19]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA to empty string for concatenation
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Create combined source and master texts
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'] + ' | Brand: ' + df['SourceBrand'] +
    ' | SubBrand: ' + df['SourceSubBrand'] + ' | MasterBrand: ' + df['SourceMasterBrand']
)

df['CombinedMasterText'] = (
    'Description: ' + df['ProductName'] + ' | Brand: ' + df['BrandName'] +
    ' | SubBrand: ' + df['SubBrandName'] + ' | MasterBrand: ' + df['MasterBrandName']
)

# Get unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()
master_texts = master_df['CombinedMasterText'].tolist()
master_products = master_df['ProductName'].tolist()

# Get all source texts
source_texts = df['CombinedSourceText'].tolist()

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Batch encode
print("Encoding master texts...")
master_embeddings = model.encode(master_texts, batch_size=2048, convert_to_tensor=True, show_progress_bar=True)

print("Encoding source texts...")
source_embeddings = model.encode(source_texts, batch_size=2048, convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarity matrix (100k x N)
print("Computing cosine similarity matrix...")
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)  # Shape: [num_sources, num_masters]

# Get best match indices and scores
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Get matched master info
df['BestMatchProduct'] = [master_products[i] for i in best_match_idx]
df['SimilarityScore'] = best_scores
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= 0.6 else 'Low')
df['IsMismatch'] = df['ProductName'] != df['BestMatchProduct']

# Optional: Join additional info from master_df
master_info = master_df.set_index('ProductName').to_dict(orient='index')
df['MatchedBrand'] = [master_info.get(p, {}).get('BrandName', '') for p in df['BestMatchProduct']]
df['MatchedSubBrand'] = [master_info.get(p, {}).get('SubBrandName', '') for p in df['BestMatchProduct']]
df['MatchedMasterBrand'] = [master_info.get(p, {}).get('MasterBrandName', '') for p in df['BestMatchProduct']]

# Save output
df.to_csv('outputofmodel_fast.csv', index=False)
print("✅ Done. Saved to 'outputofmodel_fast.csv'")


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\1255716761.py:6: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Encoding master texts...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Encoding source texts...


Batches:   0%|          | 0/51 [00:00<?, ?it/s]

Computing cosine similarity matrix...


ValueError: DataFrame index must be unique for orient='index'.

In [20]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import torch

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA to empty string
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Create combined source and master texts
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'] + ' | Brand: ' + df['SourceBrand'] +
    ' | SubBrand: ' + df['SourceSubBrand'] + ' | MasterBrand: ' + df['SourceMasterBrand']
)

df['CombinedMasterText'] = (
    'Description: ' + df['ProductName'] + ' | Brand: ' + df['BrandName'] +
    ' | SubBrand: ' + df['SubBrandName'] + ' | MasterBrand: ' + df['MasterBrandName']
)

# Get unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']] \
    .drop_duplicates(subset='ProductName')

master_texts = master_df['CombinedMasterText'].tolist()
master_products = master_df['ProductName'].tolist()

# Get source texts
source_texts = df['CombinedSourceText'].tolist()

# Load sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode master texts
print("Encoding master texts...")
master_embeddings = model.encode(master_texts, batch_size=512, convert_to_tensor=True, show_progress_bar=True)

# Encode source texts
print("Encoding source texts...")
source_embeddings = model.encode(source_texts, batch_size=512, convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarity in batches
print("Computing cosine similarity matrix in batches...")
batch_size = 512
all_best_idx = []
all_best_scores = []

for i in range(0, len(source_embeddings), batch_size):
    batch = source_embeddings[i:i + batch_size]
    cosine_scores = util.cos_sim(batch, master_embeddings)  # [batch_size, num_masters]
    best_idx = cosine_scores.argmax(dim=1).cpu().numpy()
    best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

    all_best_idx.extend(best_idx)
    all_best_scores.extend(best_scores)

# Assign match results
df['BestMatchProduct'] = [master_products[i] for i in all_best_idx]
df['SimilarityScore'] = all_best_scores
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= 0.6 else 'Low')
df['IsMismatch'] = df['ProductName'] != df['BestMatchProduct']

# Prepare a clean master info dictionary (safe indexing)
master_info = master_df.drop_duplicates(subset='ProductName') \
    .set_index('ProductName')[['BrandName', 'SubBrandName', 'MasterBrandName']] \
    .to_dict(orient='index')

# Match additional attributes
df['MatchedBrand'] = df['BestMatchProduct'].apply(lambda p: master_info.get(p, {}).get('BrandName', ''))
df['MatchedSubBrand'] = df['BestMatchProduct'].apply(lambda p: master_info.get(p, {}).get('SubBrandName', ''))
df['MatchedMasterBrand'] = df['BestMatchProduct'].apply(lambda p: master_info.get(p, {}).get('MasterBrandName', ''))

# Save output
df.to_csv('outputofmodel_fast.csv', index=False)
print("✅ Done. Saved to 'outputofmodel_fast.csv'")

C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\1796916511.py:7: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Encoding master texts...


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Encoding source texts...


Batches:   0%|          | 0/203 [00:00<?, ?it/s]

Computing cosine similarity matrix in batches...
✅ Done. Saved to 'outputofmodel_fast.csv'


In [21]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA to empty string for concatenation
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# ✅ Fix 1: Emphasize brand and subbrand more semantically
df['CombinedSourceText'] = (
    'This product is from brand ' + df['SourceBrand'] +
    ' and sub-brand ' + df['SourceSubBrand'] +
    '. Description: ' + df['SourceDescription']
)

df['CombinedMasterText'] = (
    'This product is from brand ' + df['BrandName'] +
    ' and sub-brand ' + df['SubBrandName'] +
    '. Description: ' + df['ProductName']
)

# Get unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()
master_texts = master_df['CombinedMasterText'].tolist()
master_products = master_df['ProductName'].tolist()

# Get all source texts
source_texts = df['CombinedSourceText'].tolist()

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Batch encode
print("Encoding master texts...")
master_embeddings = model.encode(master_texts, batch_size=2048, convert_to_tensor=True, show_progress_bar=True)

print("Encoding source texts...")
source_embeddings = model.encode(source_texts, batch_size=2048, convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarity matrix (100k x N)
print("Computing cosine similarity matrix...")
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)  # Shape: [num_sources, num_masters]

# Get best match indices and scores
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Get matched master info
df['BestMatchProduct'] = [master_products[i] for i in best_match_idx]
df['SimilarityScore'] = best_scores
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= 0.6 else 'Low')
df['IsMismatch'] = df['ProductName'] != df['BestMatchProduct']

# Optional: Join additional info from master_df
master_info = master_df.set_index('ProductName').to_dict(orient='index')
df['MatchedBrand'] = [master_info.get(p, {}).get('BrandName', '') for p in df['BestMatchProduct']]
df['MatchedSubBrand'] = [master_info.get(p, {}).get('SubBrandName', '') for p in df['BestMatchProduct']]
df['MatchedMasterBrand'] = [master_info.get(p, {}).get('MasterBrandName', '') for p in df['BestMatchProduct']]

# ✅ Fix 2: Downgrade confidence if brand does not match
df['IsBrandMatch'] = df['SourceBrand'].str.lower() == df['MatchedBrand'].str.lower()
df['Confidence'] = df.apply(
    lambda row: 'Low' if not row['IsBrandMatch'] else row['Confidence'],
    axis=1
)

# Save output
df.to_csv('outputofmodel_fast1.csv', index=False)
print("✅ Done. Saved to 'outputofmodel_fast1.csv'")


Encoding master texts...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Encoding source texts...


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Computing cosine similarity matrix...


ValueError: DataFrame index must be unique for orient='index'.

In [22]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA to empty string for consistency
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# ✅ Emphasize brand fields in Combined Source and Master texts
df['CombinedSourceText'] = (
    '[BRAND] ' + df['SourceBrand'] + ' [SUBBRAND] ' + df['SourceSubBrand'] +
    ' [MASTERBRAND] ' + df['SourceMasterBrand'] +
    ' [DESC] ' + df['SourceDescription']
)

df['CombinedMasterText'] = (
    '[BRAND] ' + df['BrandName'] + ' [SUBBRAND] ' + df['SubBrandName'] +
    ' [MASTERBRAND] ' + df['MasterBrandName'] +
    ' [DESC] ' + df['ProductName']
)

# Get unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()
master_texts = master_df['CombinedMasterText'].tolist()
master_products = master_df['ProductName'].tolist()

# Get all source texts
source_texts = df['CombinedSourceText'].tolist()

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Batch encode master and source texts
print("Encoding master texts...")
master_embeddings = model.encode(master_texts, batch_size=512, convert_to_tensor=True, show_progress_bar=True)

print("Encoding source texts...")
source_embeddings = model.encode(source_texts, batch_size=512, convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarity matrix
print("Computing cosine similarity matrix...")
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)  # Shape: [num_sources, num_masters]

# Get best match indices and scores
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Assign best match and score to main DataFrame
df['BestMatchProduct'] = [master_products[i] for i in best_match_idx]
df['SimilarityScore'] = best_scores
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= 0.6 else 'Low')
df['IsMismatch'] = df['ProductName'] != df['BestMatchProduct']

# Safely handle non-unique ProductName entries in master_df for merging
master_df = master_df.drop_duplicates(subset=['ProductName'])

# Map additional matched info
master_info = master_df.set_index('ProductName').to_dict(orient='index')
df['MatchedBrand'] = [master_info.get(p, {}).get('BrandName', '') for p in df['BestMatchProduct']]
df['MatchedSubBrand'] = [master_info.get(p, {}).get('SubBrandName', '') for p in df['BestMatchProduct']]
df['MatchedMasterBrand'] = [master_info.get(p, {}).get('MasterBrandName', '') for p in df['BestMatchProduct']]

# Save output
df.to_csv('outputofmodel_brand_emphasized1.csv', index=False)
print("✅ Done. Saved to 'outputofmodel_brand_emphasized1.csv'")


Encoding master texts...


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Encoding source texts...


Batches:   0%|          | 0/51 [00:00<?, ?it/s]

Computing cosine similarity matrix...
✅ Done. Saved to 'outputofmodel_brand_emphasized1.csv'


In [23]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Create enhanced inputs
df['CombinedSourceText'] = (
    '[BRAND] ' + df['SourceBrand'] + ' [SUBBRAND] ' + df['SourceSubBrand'] +
    ' [MASTERBRAND] ' + df['SourceMasterBrand'] +
    ' [DESC] ' + df['SourceDescription']
)

df['CombinedMasterText'] = (
    '[BRAND] ' + df['BrandName'] + ' [SUBBRAND] ' + df['SubBrandName'] +
    ' [MASTERBRAND] ' + df['MasterBrandName'] +
    ' [DESC] ' + df['ProductName']
)

# Unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Output columns
best_matches = []
scores = []
matched_brands = []
matched_subbrands = []
matched_masterbrands = []

print("Starting brand-restricted matching...")

for i, row in tqdm(df.iterrows(), total=len(df)):
    source_text = row['CombinedSourceText']
    source_master_brand = row['SourceMasterBrand']

    # Filter master_df by MasterBrandName
    candidate_masters = master_df[master_df['MasterBrandName'].str.lower() == source_master_brand.lower()]

    # Fallback: if no brand match, compare against all (optional)
    if candidate_masters.empty:
        candidate_masters = master_df

    # Encode
    source_embedding = model.encode(source_text, convert_to_tensor=True)
    master_embeddings = model.encode(candidate_masters['CombinedMasterText'].tolist(), convert_to_tensor=True)

    # Similarity
    cosine_scores = util.cos_sim(source_embedding, master_embeddings)[0]
    best_idx = cosine_scores.argmax().item()
    best_score = cosine_scores[best_idx].item()

    # Save result
    best_product = candidate_masters.iloc[best_idx]['ProductName']
    best_matches.append(best_product)
    scores.append(best_score)

    matched_brands.append(candidate_masters.iloc[best_idx]['BrandName'])
    matched_subbrands.append(candidate_masters.iloc[best_idx]['SubBrandName'])
    matched_masterbrands.append(candidate_masters.iloc[best_idx]['MasterBrandName'])

# Assign results
df['BestMatchProduct'] = best_matches
df['SimilarityScore'] = scores
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= 0.6 else 'Low')
df['IsMismatch'] = df['ProductName'] != df['BestMatchProduct']

df['MatchedBrand'] = matched_brands
df['MatchedSubBrand'] = matched_subbrands
df['MatchedMasterBrand'] = matched_masterbrands

# Save final output
df.to_csv('output_brand_scoped_matching1.csv', index=False)
print("✅ Matching complete. Results saved to output_brand_scoped_matching1.csv")


Starting brand-restricted matching...


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
  0%|          | 4/26000 [09:39<1046:01:16, 144.86s/it]


KeyboardInterrupt: 

In [24]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill missing values
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Create combined text columns
df['CombinedSourceText'] = (
    '[BRAND] ' + df['SourceBrand'] +
    ' [SUBBRAND] ' + df['SourceSubBrand'] +
    ' [MASTERBRAND] ' + df['SourceMasterBrand'] +
    ' [DESC] ' + df['SourceDescription']
)

df['CombinedMasterText'] = (
    '[BRAND] ' + df['BrandName'] +
    ' [SUBBRAND] ' + df['SubBrandName'] +
    ' [MASTERBRAND] ' + df['MasterBrandName'] +
    ' [DESC] ' + df['ProductName']
)

# Unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 1: Group and encode master_df by MasterBrand
master_brand_groups = {}

print("Encoding master products (just once)...")
for brand in tqdm(master_df['MasterBrandName'].unique()):
    group = master_df[master_df['MasterBrandName'] == brand]
    texts = group['CombinedMasterText'].tolist()
    embeddings = model.encode(texts, convert_to_tensor=True, batch_size=64)
    master_brand_groups[brand.lower()] = {
        'df': group.reset_index(drop=True),
        'embeddings': embeddings
    }

# Step 2: Match each source row to correct brand group
print("Running matching per row...")

results = []

for i, row in tqdm(df.iterrows(), total=len(df)):
    source_text = row['CombinedSourceText']
    source_brand = row['SourceMasterBrand'].lower()

    # Use matching group if exists, otherwise fallback to full master_df
    if source_brand in master_brand_groups:
        group_data = master_brand_groups[source_brand]
    else:
        group_data = {
            'df': master_df.reset_index(drop=True),
            'embeddings': model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True)
        }

    source_embedding = model.encode(source_text, convert_to_tensor=True)

    cosine_scores = util.cos_sim(source_embedding, group_data['embeddings'])[0]
    best_idx = torch.argmax(cosine_scores).item()
    best_score = cosine_scores[best_idx].item()

    match_row = group_data['df'].iloc[best_idx]

    results.append({
        'BestMatchProduct': match_row['ProductName'],
        'SimilarityScore': best_score,
        'Confidence': 'High' if best_score >= 0.6 else 'Low',
        'IsMismatch': row['ProductName'] != match_row['ProductName'],
        'MatchedBrand': match_row['BrandName'],
        'MatchedSubBrand': match_row['SubBrandName'],
        'MatchedMasterBrand': match_row['MasterBrandName']
    })

# Step 3: Merge results into df
results_df = pd.DataFrame(results)
df = pd.concat([df.reset_index(drop=True), results_df], axis=1)

# Save output
df.to_csv('fast_brand_scoped_output1.csv', index=False)
print("✅ Matching completed and saved.")


Encoding master products (just once)...


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 656/656 [01:30<00:00,  7.25it/s]


Running matching per row...


  0%|          | 1/26000 [02:54<1258:02:19, 174.20s/it]


KeyboardInterrupt: 

In [27]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Combine source + master text
df['CombinedSourceText'] = (
    '[BRAND] ' + df['SourceBrand'] +
    ' [SUBBRAND] ' + df['SourceSubBrand'] +
    ' [MASTERBRAND] ' + df['SourceMasterBrand'] +
    ' [DESC] ' + df['SourceDescription']
)

df['CombinedMasterText'] = (
    '[BRAND] ' + df['BrandName'] +
    ' [SUBBRAND] ' + df['SubBrandName'] +
    ' [MASTERBRAND] ' + df['MasterBrandName'] +
    ' [DESC] ' + df['ProductName']
)

# Get unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()

# Prepare output
final_results = []

# Group both source and master by MasterBrand
grouped_source = df.groupby(df['SourceMasterBrand'].str.lower())
grouped_master = master_df.groupby(master_df['MasterBrandName'].str.lower())

print("Running fast brand-scoped batched matching...\n")

# Process each MasterBrand group
for brand, source_group in tqdm(grouped_source, total=len(grouped_source)):
    master_group = grouped_master.get_group(brand) if brand in grouped_master.groups else master_df

    # Encode both in batch
    source_texts = source_group['CombinedSourceText'].tolist()
    master_texts = master_group['CombinedMasterText'].tolist()

    source_embeddings = model.encode(source_texts, convert_to_tensor=True, batch_size=128)
    master_embeddings = model.encode(master_texts, convert_to_tensor=True, batch_size=128)

    # Compute cosine similarity matrix
    cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

    # Get best match indices
    best_scores, best_indices = torch.max(cosine_scores, dim=1)

    # Prepare result rows
    for pos, (df_index, row) in enumerate(source_group.iterrows()):
        best_idx = best_indices[pos].item()
        best_score = best_scores[pos].item()
        match_row = master_group.iloc[best_idx]

        final_results.append({
            'Index': df_index,
            'OriginalProductName': row['ProductName'],
            'BestMatchProduct': match_row['ProductName'],
            'SimilarityScore': round(best_score, 4),
            'Confidence': 'High' if best_score >= 0.6 else 'Low',
            'IsMismatch': row['ProductName'] != match_row['ProductName'],
            'MatchedBrand': match_row['BrandName'],
            'MatchedSubBrand': match_row['SubBrandName'],
            'MatchedMasterBrand': match_row['MasterBrandName']
        })


# Merge back to df
results_df = pd.DataFrame(final_results).set_index('Index')
df.update(results_df)

# Save
df.to_csv("final_fast_output1.csv", index=False)
print("✅ Done. File saved as 'final_fast_output1.csv'")


Running fast brand-scoped batched matching...



C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
  1%|          | 3/370 [06:28<13:11:09, 129.34s/it]


KeyboardInterrupt: 

In [28]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Load data
df = pd.read_csv('SampleFullData.csv')

# Fill NA
for col in ['SourceDescription', 'SourceBrand', 'SourceSubBrand', 'SourceMasterBrand',
            'ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Combine source + master text
df['CombinedSourceText'] = (
    '[BRAND] ' + df['SourceBrand'] +
    ' [SUBBRAND] ' + df['SourceSubBrand'] +
    ' [MASTERBRAND] ' + df['SourceMasterBrand'] +
    ' [DESC] ' + df['SourceDescription']
)

df['CombinedMasterText'] = (
    '[BRAND] ' + df['BrandName'] +
    ' [SUBBRAND] ' + df['SubBrandName'] +
    ' [MASTERBRAND] ' + df['MasterBrandName'] +
    ' [DESC] ' + df['ProductName']
)

# Get unique master products
master_df = df[['ProductName', 'CombinedMasterText', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates()

# Prepare output
final_results = []

# Group both source and master by MasterBrand
grouped_source = df.groupby(df['SourceMasterBrand'].str.lower())
grouped_master = master_df.groupby(master_df['MasterBrandName'].str.lower())

print("Running chunked brand-scoped matching with progress bars...\n")

# Function to process chunks of source embeddings
def match_in_chunks(source_texts, master_embeddings, master_group, original_indices, batch_size=256):
    chunk_results = []
    for i in tqdm(range(0, len(source_texts), batch_size), desc=" ↳ Matching chunks", leave=False):
        chunk = source_texts[i:i+batch_size]
        chunk_embeddings = model.encode(chunk, convert_to_tensor=True, batch_size=128)
        cosine_scores = util.cos_sim(chunk_embeddings, master_embeddings)
        best_scores, best_indices = torch.max(cosine_scores, dim=1)

        for j in range(len(chunk)):
            df_index = original_indices[i + j]
            best_idx = best_indices[j].item()
            best_score = best_scores[j].item()
            match_row = master_group.iloc[best_idx]

            chunk_results.append({
                'Index': df_index,
                'BestMatchProduct': match_row['ProductName'],
                'SimilarityScore': round(best_score, 4),
                'Confidence': 'High' if best_score >= 0.6 else 'Low',
                'IsMismatch': df.loc[df_index]['ProductName'] != match_row['ProductName'],
                'MatchedBrand': match_row['BrandName'],
                'MatchedSubBrand': match_row['SubBrandName'],
                'MatchedMasterBrand': match_row['MasterBrandName']
            })
    return chunk_results

# Process each MasterBrand group
for brand, source_group in tqdm(grouped_source, desc="MasterBrand groups"):
    master_group = grouped_master.get_group(brand) if brand in grouped_master.groups else master_df

    # Precompute master embeddings
    master_texts = master_group['CombinedMasterText'].tolist()
    master_embeddings = model.encode(master_texts, convert_to_tensor=True, batch_size=128)

    # Source texts and row indices
    source_texts = source_group['CombinedSourceText'].tolist()
    source_indices = source_group.index.tolist()

    # Chunked matching
    results = match_in_chunks(source_texts, master_embeddings, master_group, source_indices)
    final_results.extend(results)

# Merge back
results_df = pd.DataFrame(final_results).set_index('Index')
df.update(results_df)

# Save
df.to_csv("final_fast_output_chunked.csv", index=False)
print("✅ Done. Output saved as 'final_fast_output_chunked.csv'")


Running chunked brand-scoped matching with progress bars...



C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)

Matching chunks:   0%|          | 0/1 [00:00<?, ?it/s]
                                                      
Matching chunks:   0%|          | 0/1 [00:00<?, ?it/s]
                                                      
MasterBrand groups:   2%|▏         | 7/370 [12:18<10:38:02, 105.46s/it]


KeyboardInterrupt: 

In [29]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in helper columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']:
    df[col] = df[col].fillna('')

# Create enriched text for raw data
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'].astype(str) + ' | ' +
    'Brand: ' + df['SourceBrand'] + ' | ' +
    'SubBrand: ' + df['SourceSubBrand'] + ' | ' +
    'MasterBrand: ' + df['SourceMasterBrand']
)

# Create master list with additional brand info
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')

# Create enriched master product text
master_df['CombinedMasterText'] = (
    'Description: ' + master_df['ProductName'].astype(str) + ' | ' +
    'Brand: ' + master_df['BrandName'] + ' | ' +
    'SubBrand: ' + master_df['SubBrandName'] + ' | ' +
    'MasterBrand: ' + master_df['MasterBrandName']
)

# Encode with SBERT
model = SentenceTransformer('all-MiniLM-L6-v2')
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True)
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Get best matches
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Add matched info to df
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save to CSV
df.to_csv('outputofmodel2.csv', index=False)

# Preview
print("✅ Matching complete. Output saved to 'outputofmodel2.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence']].head())


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Matching complete. Output saved to 'outputofmodel2.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

                BestMatchProduct  SimilarityScore Confidence  
0                    CURL MOUSSE         0.857571       High  
1                    CURL MOUSSE         0.828772       High  
2      CURLS FIRM STYLING MOUSSE         0.821535       High  
3        10 IN 1 CREAM IN MOUSSE         0.766348       High  
4  NATURAL CURL ENHANCING MOUSSE         0.820982       High  


In [30]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related and product columns with empty strings
brand_cols_source = ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']
brand_cols_master = ['BrandName', 'SubBrandName', 'MasterBrandName']
for col in brand_cols_source + brand_cols_master:
    df[col] = df[col].fillna('').astype(str)

# Create BrandContext for source and master
df['BrandContext'] = (
    df['SourceBrand'].str.strip() + ' ' +
    df['SourceSubBrand'].str.strip() + ' ' +
    df['SourceMasterBrand'].str.strip()
).str.strip()

# Create enriched text for raw data
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'].astype(str).str.strip() + ' | ' +
    'BrandContext: ' + df['BrandContext']
)

# Create master product list with deduplication
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().copy()

# Fill missing brand data in master
for col in ['BrandName', 'SubBrandName', 'MasterBrandName']:
    master_df[col] = master_df[col].fillna('').astype(str)

# Create BrandContext in master
master_df['BrandContext'] = (
    master_df['BrandName'].str.strip() + ' ' +
    master_df['SubBrandName'].str.strip() + ' ' +
    master_df['MasterBrandName'].str.strip()
).str.strip()

# Create enriched master product text
master_df['CombinedMasterText'] = (
    'Description: ' + master_df['ProductName'].astype(str).str.strip() + ' | ' +
    'BrandContext: ' + master_df['BrandContext']
)

# Load model
print("🔄 Loading SBERT model...")
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode with progress bar
print("🔄 Encoding source texts...")
source_texts = df['CombinedSourceText'].tolist()
source_embeddings = model.encode(source_texts, convert_to_tensor=True, batch_size=128, show_progress_bar=True)

print("🔄 Encoding master texts...")
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, batch_size=128, show_progress_bar=True)

# Compute cosine similarities
print("🔄 Computing cosine similarity matrix...")
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Get best matches
print("🔄 Extracting best matches...")
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Add matched info to df
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save to CSV
df.to_csv('outputofmodel3.csv', index=False)

# Preview
print("✅ Matching complete. Output saved to 'outputofmodel3.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence']].head())


🔄 Loading SBERT model...
🔄 Encoding source texts...


Batches:   0%|          | 0/204 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


🔄 Encoding master texts...


Batches:   0%|          | 0/80 [00:00<?, ?it/s]

🔄 Computing cosine similarity matrix...
🔄 Extracting best matches...
✅ Matching complete. Output saved to 'outputofmodel3.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

            BestMatchProduct  SimilarityScore Confidence  
0                CURL MOUSSE         0.804365       High  
1                CURL MOUSSE         0.820288       High  
2  CURLS FIRM STYLING MOUSSE         0.807076       High  
3    10 IN 1 CREAM IN MOUSSE         0.756025       High  
4                CURL MOUSSE         0.802275       High  


In [31]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing required text fields
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']:
    df[col] = df[col].fillna('')

# Create BrandContext
df['BrandContext'] = df['SourceMasterBrand'].astype(str) + " " + df['SourceBrand'] + " " + df['SourceSubBrand']
df['BrandContext'] = df['BrandContext'].str.strip()

# Create CombinedSourceText with brand emphasis
df['CombinedSourceText'] = (
    'BrandContext: ' + df['BrandContext'] + ' | ' +
    'Description: ' + df['SourceDescription'].astype(str) + ' | ' +
    'BrandContext: ' + df['BrandContext']  # repeated to give more weight
)

# Create master list with unique product records
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')

# Create master BrandContext
master_df['BrandContext'] = master_df['MasterBrandName'].astype(str) + " " + master_df['BrandName'] + " " + master_df['SubBrandName']
master_df['BrandContext'] = master_df['BrandContext'].str.strip()

# Create CombinedMasterText
master_df['CombinedMasterText'] = (
    'BrandContext: ' + master_df['BrandContext'] + ' | ' +
    'Description: ' + master_df['ProductName'].astype(str) + ' | ' +
    'BrandContext: ' + master_df['BrandContext']
)

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode with progress bars
print("🔍 Encoding source records...")
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

print("📦 Encoding master products...")
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarity
print("📊 Computing cosine similarities...")
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Get best match index and score
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Attach matched info
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Set confidence flag
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Brand consistency logic
def is_brand_consistent(source_row, match_row):
    return (
        source_row['SourceBrand'].strip().lower() == match_row['BrandName'].strip().lower()
        or source_row['SourceSubBrand'].strip().lower() == match_row['SubBrandName'].strip().lower()
        or source_row['SourceMasterBrand'].strip().lower() == match_row['MasterBrandName'].strip().lower()
    )

# Check brand consistency row-wise
print("🔍 Checking brand consistency...")
df['BrandConsistent'] = [
    is_brand_consistent(source_row, match_row)
    for (_, source_row), (_, match_row) in tqdm(zip(df.iterrows(), master_df.iloc[best_match_idx].iterrows()), total=len(df))
]

# Final accuracy flag
df['Accuracy'] = df['BrandConsistent']

# Save output
df.to_csv('outputofmodel4', index=False)
print("✅ Matching complete. Output saved to 'outputofmodel4.csv'")

# Preview a few records
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence', 'Accuracy']].head())


🔍 Encoding source records...


Batches:   0%|          | 0/813 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


📦 Encoding master products...


Batches:   0%|          | 0/319 [00:00<?, ?it/s]

📊 Computing cosine similarities...
🔍 Checking brand consistency...


100%|██████████| 25999/25999 [00:02<00:00, 9364.54it/s] 


✅ Matching complete. Output saved to 'outputofmodel4.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

            BestMatchProduct  SimilarityScore Confidence  Accuracy  
0    10 IN 1 CREAM IN MOUSSE         0.793987       High     False  
1    10 IN 1 CREAM IN MOUSSE         0.864063       High     False  
2  CURLS FIRM STYLING MOUSSE         0.781853       High     False  
3    10 IN 1 CREAM IN MOUSSE         0.848228       High      True  
4    10 IN 1 CREAM IN MOUSSE         0.817913       High     False  


In [32]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from tqdm import tqdm

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing required text fields
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']:
    df[col] = df[col].fillna('')

# Create BrandContext
df['BrandContext'] = df['SourceMasterBrand'].astype(str) + " " + df['SourceBrand'] + " " + df['SourceSubBrand']
df['BrandContext'] = df['BrandContext'].str.strip()

# Create CombinedSourceText with brand emphasis
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'].astype(str) + ' | ' +
    'BrandContext: ' + df['BrandContext']
)

# Create master list with unique product records
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')

# Create master BrandContext
master_df['BrandContext'] = master_df['MasterBrandName'].astype(str) + " " + master_df['BrandName'] + " " + master_df['SubBrandName']
master_df['BrandContext'] = master_df['BrandContext'].str.strip()

# Create CombinedMasterText
master_df['CombinedMasterText'] = (
    'Description: ' + master_df['ProductName'].astype(str) + ' | ' +
    'BrandContext: ' + master_df['BrandContext']
)

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode with progress bars
print("🔍 Encoding source records...")
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

print("📦 Encoding master products...")
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarity
print("📊 Computing cosine similarities...")
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Get best match index and score
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Attach matched info
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Set confidence flag
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Brand consistency logic
def is_brand_consistent(source_row, match_row):
    return (
        source_row['SourceBrand'].strip().lower() == match_row['BrandName'].strip().lower()
        or source_row['SourceSubBrand'].strip().lower() == match_row['SubBrandName'].strip().lower()
        or source_row['SourceMasterBrand'].strip().lower() == match_row['MasterBrandName'].strip().lower()
    )

# Check brand consistency row-wise
print("🔍 Checking brand consistency...")
df['BrandConsistent'] = [
    is_brand_consistent(source_row, match_row)
    for (_, source_row), (_, match_row) in tqdm(zip(df.iterrows(), master_df.iloc[best_match_idx].iterrows()), total=len(df))
]

# Final accuracy flag
df['Accuracy'] = df['BrandConsistent']

# Save output
df.to_csv('outputofmodel5', index=False)
print("✅ Matching complete. Output saved to 'outputofmodel5.csv'")

# Preview a few records
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence', 'Accuracy']].head())


🔍 Encoding source records...


Batches:   0%|          | 0/813 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


📦 Encoding master products...


Batches:   0%|          | 0/319 [00:00<?, ?it/s]

📊 Computing cosine similarities...
🔍 Checking brand consistency...


100%|██████████| 25999/25999 [00:02<00:00, 8702.32it/s] 


✅ Matching complete. Output saved to 'outputofmodel5.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

            BestMatchProduct  SimilarityScore Confidence  Accuracy  
0                CURL MOUSSE         0.804365       High     False  
1                CURL MOUSSE         0.820288       High     False  
2  CURLS FIRM STYLING MOUSSE         0.807811       High     False  
3    10 IN 1 CREAM IN MOUSSE         0.769096       High      True  
4                CURL MOUSSE         0.802275       High     False  


In [33]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand']:
    df[col] = df[col].fillna('')

# Create CombinedSourceText (no repeated brand terms)
df['CombinedSourceText'] = (
    'Description: ' + df['SourceDescription'].astype(str) + ' | ' +
    'BrandContext: ' + df['SourceBrand'] + ' ' + df['SourceSubBrand'] + ' ' + df['SourceMasterBrand']
)

# Prepare Master Data
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = (
    'Description: ' + master_df['ProductName'].astype(str) + ' | ' +
    'BrandContext: ' + master_df['BrandName'] + ' ' + master_df['SubBrandName'] + ' ' + master_df['MasterBrandName']
)

# Load SBERT Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode texts
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True)
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Find best match index & score for each row
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Append best match results
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Rule-based accuracy tagging based on brand or masterbrand
def check_brand_accuracy(row):
    source_brand = row['SourceBrand'].strip().lower()
    matched_brand = row['MatchedBrand'].strip().lower()
    source_master = row['SourceMasterBrand'].strip().lower()
    matched_master = row['MatchedMasterBrand'].strip().lower()

    if source_brand and matched_brand and source_brand == matched_brand:
        return 'TRUE'
    elif source_master and matched_master and source_master == matched_master:
        return 'TRUE'
    else:
        return 'FALSE'

df['Accuracy'] = df.apply(check_brand_accuracy, axis=1)

# Save output
df.to_csv('outputofmodel6.csv', index=False)

# Preview
print("✅ Matching complete. Output saved to 'outputofmodel6.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence', 'Accuracy']].head())


C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


✅ Matching complete. Output saved to 'outputofmodel6.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

            BestMatchProduct  SimilarityScore Confidence Accuracy  
0                CURL MOUSSE         0.804365       High    FALSE  
1                CURL MOUSSE         0.820288       High    FALSE  
2  CURLS FIRM STYLING MOUSSE         0.807076       High    FALSE  
3    10 IN 1 CREAM IN MOUSSE         0.756025       High    FALSE  
4                CURL MOUSSE         0.802275       High    FALSE  


In [34]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# 🧠 STEP 1: Integrate brand fields into SourceDescription and ProductName
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# Prepare Master Data with enriched descriptions
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# Load SBERT Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode texts with progress bar
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True, show_progress_bar=True)
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Find best match index & score for each row
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Append best match results
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.6
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Rule-based accuracy tagging based on brand or masterbrand
def check_brand_accuracy(row):
    source_brand = row['SourceBrand'].strip().lower()
    matched_brand = row['MatchedBrand'].strip().lower()
    source_master = row['SourceMasterBrand'].strip().lower()
    matched_master = row['MatchedMasterBrand'].strip().lower()

    if source_brand and matched_brand and source_brand == matched_brand:
        return 'TRUE'
    elif source_master and matched_master and source_master == matched_master:
        return 'TRUE'
    else:
        return 'FALSE'

df['Accuracy'] = df.apply(check_brand_accuracy, axis=1)

# Save output
df.to_csv('outputofmodel7.csv', index=False)

# Preview
print("✅ Step 1 complete. Output saved to 'outputofmodel7.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence', 'Accuracy']].head())


Batches:   0%|          | 0/813 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/319 [00:00<?, ?it/s]

✅ Step 1 complete. Output saved to 'outputofmodel7.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

          BestMatchProduct  SimilarityScore Confidence Accuracy  
0  10 IN 1 CREAM IN MOUSSE         0.763084       High    FALSE  
1  10 IN 1 CREAM IN MOUSSE         0.755397       High    FALSE  
2        CURLS WHIP MOUSSE         0.776402       High    FALSE  
3  10 IN 1 CREAM IN MOUSSE         0.762371       High    FALSE  
4              CURL MOUSSE         0.738674       High    FALSE  


In [35]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# 🧠 STEP 1: Integrate brand fields into SourceDescription and ProductName
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# Prepare Master Data with enriched descriptions
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# Load SBERT Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode texts with progress bar
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True, show_progress_bar=True)
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Find best match index & score for each row
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Append best match results
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.7
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save output
df.to_csv('outputofmodel8.csv', index=False)

# Preview
print("✅ Step 1 complete. Output saved to 'outputofmodel8.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence', 'Accuracy']].head())


Batches:   0%|          | 0/813 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/319 [00:00<?, ?it/s]

✅ Step 1 complete. Output saved to 'outputofmodel8.csv'
                   SourceDescription              ProductName  \
0             Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
1      Loreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   
2               Curls 10 In 1 Mousse  10 IN 1 CREAM IN MOUSSE   
3  SOIN MULTI BENEFICES CREME-MOUSSE  10 IN 1 CREAM IN MOUSSE   
4     L'Oreal Curl Expression Mousse  10 IN 1 CREAM IN MOUSSE   

          BestMatchProduct  SimilarityScore Confidence Accuracy  
0  10 IN 1 CREAM IN MOUSSE         0.763084       High    FALSE  
1  10 IN 1 CREAM IN MOUSSE         0.755397       High    FALSE  
2        CURLS WHIP MOUSSE         0.776402       High    FALSE  
3  10 IN 1 CREAM IN MOUSSE         0.762371       High    FALSE  
4              CURL MOUSSE         0.738674       High    FALSE  


In [36]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Replace NaNs in brand-related columns with empty strings
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# 🧠 STEP 1: Integrate brand fields into SourceDescription and ProductName
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# Prepare Master Data with enriched descriptions
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# Load SBERT Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode texts with progress bar
source_embeddings = model.encode(df['CombinedSourceText'].tolist(), convert_to_tensor=True, show_progress_bar=True)
master_embeddings = model.encode(master_df['CombinedMasterText'].tolist(), convert_to_tensor=True, show_progress_bar=True)

# Compute cosine similarities
cosine_scores = util.cos_sim(source_embeddings, master_embeddings)

# Find best match index & score for each row
best_match_idx = cosine_scores.argmax(dim=1).cpu().numpy()
best_scores = cosine_scores.max(dim=1).values.cpu().numpy()

# Append best match results
df['BestMatchProduct'] = master_df.iloc[best_match_idx]['ProductName'].values
df['MatchedBrand'] = master_df.iloc[best_match_idx]['BrandName'].values
df['MatchedSubBrand'] = master_df.iloc[best_match_idx]['SubBrandName'].values
df['MatchedMasterBrand'] = master_df.iloc[best_match_idx]['MasterBrandName'].values
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.7
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save output
df.to_csv('outputofmodel9.csv', index=False)

# Preview
print("✅ Step 1 complete. Output saved to 'outputofmodel9.csv'")
print(df[['SourceDescription', 'ProductName', 'BestMatchProduct', 'SimilarityScore', 'Confidence', 'Accuracy']].head())


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\2808660426.py:5: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Batches:   0%|          | 0/10532 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/1130 [00:00<?, ?it/s]

RuntimeError: [enforce fail at alloc_cpu.cpp:116] data. DefaultCPUAllocator: not enough memory: you tried to allocate 48703384520 bytes.

In [37]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm

# Load CSV
df = pd.read_csv('SampleFullData.csv')

# Drop rows with missing SourceDescription or ProductName
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# Fill NaNs in brand columns
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# Enrich with brand info
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# Master Data
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# Load SBERT Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode master data once
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# Prepare for batching
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

# Batch-wise matching
for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # shape [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# Add match results
df = df.iloc[:len(best_scores)].copy()  # ensure same length

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]
df['SimilarityScore'] = best_scores

# Confidence tagging
threshold = 0.7
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# Save results
df.to_csv('output_batched_model9.csv', index=False)

print("✅ Batched matching complete! Results saved to 'output_batched_model9.csv'")


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\3471367738.py:7: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Batches:   0%|          | 0/1130 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 68/68 [27:14<00:00, 24.04s/it]


✅ Batched matching complete! Results saved to 'output_batched_model9.csv'


In [40]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz import fuzz

# === Load CSV ===
df = pd.read_csv('SampleLessData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy string match (normalized to 0-1)
def fuzzy_overlap(a, b):
    return fuzz.partial_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores
alpha = 0.7  # SBERT weight
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# Confidence tagging
threshold = 0.7
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model10.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model10.csv'")


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 1/1 [00:08<00:00,  8.58s/it]


✅ Matching complete! Results saved to 'output_combined_model10.csv'


In [39]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ------------------- -------------------- 0.8/1.6 MB 7.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 6.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 6.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 6.7 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 1.4 MB/s eta 0:00:00


In [41]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz import fuzz

# === Load CSV ===
df = pd.read_csv('SampleFullData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy string match (normalized to 0-1)
def fuzzy_overlap(a, b):
    return fuzz.partial_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores
alpha = 0.7  # SBERT weight
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# Confidence tagging
threshold = 0.7
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model10.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model10.csv'")


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\1048795411.py:8: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


KeyboardInterrupt: 

In [42]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz import fuzz

# === Load CSV ===
df = pd.read_csv('SampleFullData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy string match (normalized to 0-1)
def fuzzy_overlap(a, b):
    return fuzz.partial_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores
alpha = 0.7  # SBERT weight
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# Confidence tagging
threshold = 0.7
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model11.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model11.csv'")


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\3427055514.py:8: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Batches:   0%|          | 0/1130 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 68/68 [27:53<00:00, 24.61s/it]


✅ Matching complete! Results saved to 'output_combined_model11.csv'


In [45]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('SampleFullData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Improved fuzzy matching using token_set_ratio
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores: adjust alpha
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# Confidence tagging
threshold = 0.65
df['Confidence'] = df['SimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model14.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model14.csv'")


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\2030125212.py:8: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Batches:   0%|          | 0/1130 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 68/68 [28:19<00:00, 25.00s/it]


✅ Matching complete! Results saved to 'output_combined_model14.csv'


In [48]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('SampleLessData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.10, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.08, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model17.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model1.csv'")


C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\801549580.py:8: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Batches:   0%|          | 0/1130 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 68/68 [32:36<00:00, 28.76s/it]


✅ Matching complete! Results saved to 'output_combined_model17.csv'


In [51]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('SampleFullData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.50, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.03, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model18.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model18.csv'")
     

C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\2921469059.py:8: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')
Exception ignored in: 'zmq.backend.cython._zmq.Frame.__del__'
Traceback (most recent call last):
  File "_zmq.py", line 160, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


KeyboardInterrupt: 

In [52]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('SampleFullData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.50, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.03, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model19.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model19.csv'")
     

C:\Users\Sarthak.Prakash\AppData\Local\Temp\ipykernel_3288\3896107411.py:8: DtypeWarning: Columns (1,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SampleFullData.csv')


Batches:   0%|          | 0/1130 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 68/68 [29:09<00:00, 25.73s/it]


✅ Matching complete! Results saved to 'output_combined_model19.csv'


In [53]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('HackWeekProductsData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.50, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.03, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model20.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model20.csv'")
     

Batches:   0%|          | 0/1506 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


KeyboardInterrupt: 

In [54]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('FullData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.50, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.03, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
df.to_csv('output_combined_model20.csv', index=False)

print("✅ Matching complete! Results saved to 'output_combined_model20.csv'")

Batches:   0%|          | 0/1506 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 151/151 [1:06:27<00:00, 26.41s/it]


✅ Matching complete! Results saved to 'output_combined_model20.csv'


In [55]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('FullDataTest.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.50, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.03, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
# Select only the desired output columns
output_df = df[['ServiceAndProductMappingId', 'ProductMasterId', 'ProductName', 'BestMatchProduct', 'AdjustedSimilarityScore', 'Confidence']]

# Save to CSV
output_df.to_csv('output_combined_model21.csv', index=False)

print("✅ Matching complete! Filtered results saved to 'output_combined_model21.csv'")

Batches:   0%|          | 0/1491 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 145/145 [1:03:24<00:00, 26.24s/it]


✅ Matching complete! Filtered results saved to 'output_combined_model21.csv'


In [56]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from tqdm import tqdm
from rapidfuzz.fuzz import token_set_ratio

# === Load CSV ===
df = pd.read_csv('FinalData.csv')

# === Drop rows with missing SourceDescription or ProductName ===
df = df.dropna(subset=['SourceDescription', 'ProductName'])

# === Fill NaNs in brand columns ===
for col in ['SourceBrand', 'SourceSubBrand', 'SourceMasterBrand', 'BrandName', 'SubBrandName', 'MasterBrandName']:
    df[col] = df[col].fillna('')

# === Function to enrich with brand info ===
def enrich_with_brand(desc, brand, subbrand, masterbrand):
    parts = [brand, subbrand, masterbrand]
    brand_str = ' '.join([p.strip() for p in parts if p.strip()])
    return f"{brand_str} {desc}".strip()

# === Combined Source Text ===
df['CombinedSourceText'] = df.apply(
    lambda row: f"Description: {enrich_with_brand(row['SourceDescription'], row['SourceBrand'], row['SourceSubBrand'], row['SourceMasterBrand'])}",
    axis=1
)

# === Prepare Master Data ===
master_df = df[['ProductName', 'BrandName', 'SubBrandName', 'MasterBrandName']].drop_duplicates().fillna('')
master_df['CombinedMasterText'] = master_df.apply(
    lambda row: f"Description: {enrich_with_brand(row['ProductName'], row['BrandName'], row['SubBrandName'], row['MasterBrandName'])}",
    axis=1
)

# === Load SBERT Model ===
model = SentenceTransformer('all-MiniLM-L6-v2')

# === Encode Master Texts ===
master_texts = master_df['CombinedMasterText'].tolist()
master_embeddings = model.encode(master_texts, convert_to_tensor=True, show_progress_bar=True)

# === Batched Matching ===
batch_size = 5000
source_texts = df['CombinedSourceText'].tolist()

best_indices = []
best_scores = []

for i in tqdm(range(0, len(source_texts), batch_size), desc="Matching"):
    batch_texts = source_texts[i:i+batch_size]
    batch_embeddings = model.encode(batch_texts, convert_to_tensor=True, show_progress_bar=False)
    
    scores = util.cos_sim(batch_embeddings, master_embeddings)  # [batch_size x M]
    best_batch_scores, best_batch_indices = scores.max(dim=1)

    best_scores.extend(best_batch_scores.cpu().numpy())
    best_indices.extend(best_batch_indices.cpu().numpy())

# === Add Matching Results ===
df = df.iloc[:len(best_scores)].copy()  # ensure alignment

df['BestMatchProduct'] = [master_df.iloc[i]['ProductName'] for i in best_indices]
df['MatchedBrand'] = [master_df.iloc[i]['BrandName'] for i in best_indices]
df['MatchedSubBrand'] = [master_df.iloc[i]['SubBrandName'] for i in best_indices]
df['MatchedMasterBrand'] = [master_df.iloc[i]['MasterBrandName'] for i in best_indices]

# === Score Calculation ===

# SBERT score
df['SBERTScore'] = best_scores

# Fuzzy match (ProductNameScore)
def fuzzy_overlap(a, b):
    return token_set_ratio(str(a), str(b)) / 100.0

df['ProductNameScore'] = df.apply(
    lambda row: fuzzy_overlap(row['SourceDescription'], row['BestMatchProduct']),
    axis=1
)

# Combine scores using alpha weight
alpha = 0.8  # More weight to SBERT for precision
df['SimilarityScore'] = df.apply(
    lambda row: alpha * row['SBERTScore'] + (1 - alpha) * row['ProductNameScore'],
    axis=1
)

# === Adjust similarity based on fuzzy score ===
def adjust_similarity(row):
    sim = row['SimilarityScore']
    fuzzy = row['ProductNameScore']
    
    if fuzzy >= 0.90:
        return min(sim + 0.50, 1.0)
    elif fuzzy <= 0.30:
        return max(sim - 0.03, 0.0)
    else:
        return sim

df['AdjustedSimilarityScore'] = df.apply(adjust_similarity, axis=1)

# === Confidence tagging based on adjusted score ===
threshold = 0.65
df['Confidence'] = df['AdjustedSimilarityScore'].apply(lambda x: 'High' if x >= threshold else 'Low')

# === Save Output ===
# Select only the desired output columns
output_df = df[['ServiceAndProductMappingId', 'ProductMasterId', 'ProductName', 'BestMatchProduct', 'AdjustedSimilarityScore', 'Confidence']]

# Save to CSV
output_df.to_csv('FinalOutput.csv', index=False)

print("✅ Matching complete! Filtered results saved to 'FinalOutput.csv'")

Batches:   0%|          | 0/1491 [00:00<?, ?it/s]

C:\Users\Sarthak.Prakash\AppData\Local\anaconda3\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Matching: 100%|██████████| 145/145 [1:02:34<00:00, 25.89s/it]


✅ Matching complete! Filtered results saved to 'FinalOutput.csv'
